In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Reshape, Softmax, Layer, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, ResNet50, EfficientNetB0
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, accuracy_score

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory("data_large/Training",
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode='categorical',
                                               subset='training')

val_data = val_datagen.flow_from_directory("data_large/Training",
                                           target_size=(224, 224),
                                           batch_size=32,
                                           class_mode='categorical',
                                           subset='validation')

test_data = test_datagen.flow_from_directory("data_large/Testing",
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode='categorical',
                                             shuffle=False)

input_shape = (224, 224, 3)
num_classes = train_data.num_classes

Found 4571 images belonging to 4 classes.
Found 1141 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [9]:
print(train_data.class_indices)
print(val_data.class_indices)
print(test_data.class_indices)

{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


In [3]:
#CNN

model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation="softmax")
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
72/72 [==============================] - 24s 271ms/step - loss: 2.7661 - accuracy: 0.4802 - val_loss: 9.5807 - val_accuracy: 0.2880
Epoch 2/10
72/72 [==============================] - 17s 229ms/step - loss: 0.9405 - accuracy: 0.6413 - val_loss: 11.1500 - val_accuracy: 0.2880
Epoch 3/10
72/72 [==============================] - 18s 245ms/step - loss: 0.8943 - accuracy: 0.6387 - val_loss: 7.7676 - val_accuracy: 0.2914
Epoch 4/10
72/72 [==============================] - 18s 243ms/step - loss: 0.8341 - accuracy: 0.6652 - val_loss: 3.6061 - val_accuracy: 0.3508
Epoch 5/10
72/72 [==============================] - 17s 233ms/step - loss: 0.7814 - accuracy: 0.6839 - val_loss: 5.2824 - val_accuracy: 0.2949
Epoch 6/10
72/72 [==============================] - 17s 233ms/step - loss: 0.7411 - accuracy: 0.6939 - val_loss: 3.9133 - val_accuracy: 0.2042
Epoch 7/10
72/72 [==============================] - 17s 235ms/step - loss: 0.7169 - accuracy: 0.7088 - val_loss: 2.1793 - val_accuracy: 0.357

In [ ]:
# Defining the basic model architecture
def create_model(input_shape, num_classes, learning_rate):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation="softmax")
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Greedy Feature Selection and Hyperparameter Search

learning_rates = [0.0001, 0.001, 0.01]
best_acc = 0
best_lr = 0

for lr in learning_rates:
    model = create_model(input_shape=input_shape, num_classes=num_classes, learning_rate=lr)
    history = model.fit(train_data, epochs=10, validation_data=val_data)
    test_loss, test_acc = model.evaluate(test_data)
    
    print(f"Learning Rate: {lr}, Test Accuracy: {test_acc:.4f}")
    
    if test_acc > best_acc:
        best_acc = test_acc
        best_lr = lr

print(f"Best Learning Rate: {best_lr} with Test Accuracy: {best_acc:.4f}")

Epoch 1/10
143/143 [==============================] - 41s 273ms/step - loss: 1.5473 - accuracy: 0.6165 - val_loss: 8.6996 - val_accuracy: 0.2726
Epoch 2/10
143/143 [==============================] - 35s 244ms/step - loss: 0.7206 - accuracy: 0.7184 - val_loss: 3.1272 - val_accuracy: 0.5188
Epoch 3/10
143/143 [==============================] - 34s 237ms/step - loss: 0.6224 - accuracy: 0.7626 - val_loss: 2.8683 - val_accuracy: 0.3637
Epoch 4/10
143/143 [==============================] - 34s 237ms/step - loss: 0.5602 - accuracy: 0.7828 - val_loss: 0.6189 - val_accuracy: 0.7748
Epoch 5/10
143/143 [==============================] - 35s 244ms/step - loss: 0.5131 - accuracy: 0.8025 - val_loss: 0.6905 - val_accuracy: 0.7607
Epoch 6/10
143/143 [==============================] - 34s 236ms/step - loss: 0.4962 - accuracy: 0.8060 - val_loss: 0.8556 - val_accuracy: 0.7616
Epoch 7/10
143/143 [==============================] - 34s 238ms/step - loss: 0.4566 - accuracy: 0.8280 - val_loss: 1.5465 - val_ac

In [ ]:
# Function to create a model with varying architecture
def create_model_varying(input_shape, num_classes, filters, kernel_size, dense_units, dropout_rate):
    model = Sequential([
        Conv2D(filters, kernel_size=kernel_size, activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        Conv2D(filters*2, kernel_size=kernel_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        Conv2D(filters*4, kernel_size=kernel_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        GlobalAveragePooling2D(),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(num_classes, activation="softmax")
    ])
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Varying Architecture
filters_options = [32, 64, 128]
kernel_sizes = [(3, 3), (5, 5)]
dense_units_options = [256, 512]
dropout_options = [0.3, 0.5]
best_acc = 0
best_config = {}

# Perform Greedy Feature Selection
for filters in filters_options:
    for kernel_size in kernel_sizes:
        for dense_units in dense_units_options:
            for dropout_rate in dropout_options:
                print(f"Training with {filters} filters, kernel_size {kernel_size}, {dense_units} dense units, dropout {dropout_rate}")
                model = create_model_varying(input_shape=(224, 224, 3), num_classes=4, filters=filters, kernel_size=kernel_size, dense_units=dense_units, dropout_rate=dropout_rate)
                model.fit(train_data, epochs=5, validation_data=val_data)
                test_loss, test_acc = model.evaluate(test_data)
                
                print(f"Test Accuracy: {test_acc:.4f}")
                
                if test_acc > best_acc:
                    best_acc = test_acc
                    best_config = {
                        "filters": filters,
                        "kernel_size": kernel_size,
                        "dense_units": dense_units,
                        "dropout_rate": dropout_rate
                    }
                
                tf.keras.backend.clear_session()

print(f"Best Configuration: {best_config}")
print(f"Best Test Accuracy: {best_acc:.4f}")

Training with 32 filters, kernel_size (3, 3), 256 dense units, dropout 0.3
Epoch 1/5
143/143 [==============================] - 37s 249ms/step - loss: 0.7735 - accuracy: 0.6939 - val_loss: 4.0943 - val_accuracy: 0.2550
Epoch 2/5
143/143 [==============================] - 34s 240ms/step - loss: 0.5971 - accuracy: 0.7709 - val_loss: 4.2274 - val_accuracy: 0.2901
Epoch 3/5
143/143 [==============================] - 34s 235ms/step - loss: 0.5158 - accuracy: 0.8000 - val_loss: 7.6811 - val_accuracy: 0.2621
Epoch 4/5
143/143 [==============================] - 34s 236ms/step - loss: 0.4521 - accuracy: 0.8333 - val_loss: 1.0321 - val_accuracy: 0.6284
Epoch 5/5
41/41 [==============================] - 2s 41ms/step - loss: 1.0403 - accuracy: 0.6560
Test Accuracy: 0.6560
Training with 32 filters, kernel_size (3, 3), 256 dense units, dropout 0.5
Epoch 1/5
143/143 [==============================] - 34s 237ms/step - loss: 0.7989 - accuracy: 0.6756 - val_loss: 2.1732 - val_accuracy: 0.2577
Epoch 2/5


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential/batch_normalization/FusedBatchNormGradV3' defined at (most recent call last):
    File "c:\Users\Jayan\anaconda3\envs\ai\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Jayan\anaconda3\envs\ai\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\Jayan\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Jayan\anaconda3\envs\ai\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\Jayan\anaconda3\envs\ai\lib\site-packages\ipykernel\kernelapp.py", line 701, in start
      self.io_loop.start()
    File "c:\Users\Jayan\anaconda3\envs\ai\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\J

In [ ]:
# Function to create a model with varying hyperparameters
def create_model_with_hyperparameters(input_shape, num_classes, learning_rate, batch_size, filters, kernel_size, dense_units, dropout_rate):
    model = Sequential([
        Conv2D(filters, kernel_size=kernel_size, activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        Conv2D(filters*2, kernel_size=kernel_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        Conv2D(filters*4, kernel_size=kernel_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2,2)),

        Flatten(),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(num_classes, activation="softmax")
    ])
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Hyperparameter options
learning_rates = [0.0001, 0.001, 0.01]
batch_sizes = [32, 64, 128]
filters_options = [32, 64]
kernel_sizes = [(3, 3), (5, 5)]
dense_units_options = [256, 512]
dropout_options = [0.3, 0.5]

best_acc = 0
best_hyperparameters = {}

# Perform Greedy Hyperparameter Search
for lr in learning_rates:
    for batch_size in batch_sizes:
        for filters in filters_options:
            for kernel_size in kernel_sizes:
                for dense_units in dense_units_options:
                    for dropout_rate in dropout_options:
                        print(f"Training with lr={lr}, batch_size={batch_size}, {filters} filters, kernel_size={kernel_size}, {dense_units} dense units, dropout={dropout_rate}")
                        
                        # Create the model with the current hyperparameters
                        model = create_model_with_hyperparameters(input_shape=(224, 224, 3), num_classes=10, learning_rate=lr, batch_size=batch_size, filters=filters, kernel_size=kernel_size, dense_units=dense_units, dropout_rate=dropout_rate)
                        
                        model.fit(train_data, epochs=5, batch_size=batch_size, validation_data=val_data)
                        test_loss, test_acc = model.evaluate(test_data)
                        
                        print(f"Test Accuracy: {test_acc:.4f}")
                        
                        if test_acc > best_acc:
                            best_acc = test_acc
                            best_hyperparameters = {
                                "learning_rate": lr,
                                "batch_size": batch_size,
                                "filters": filters,
                                "kernel_size": kernel_size,
                                "dense_units": dense_units,
                                "dropout_rate": dropout_rate
                            }

print(f"Best Hyperparameters: {best_hyperparameters}")
print(f"Best Test Accuracy: {best_acc:.4f}")

In [ ]:
# Define the CNN model
def create_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation="softmax")
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Modify model to output features from the last convolutional layer
def create_feature_extractor(input_shape):
    model = create_model(input_shape, num_classes=10)  # Modify num_classes according to your dataset
    feature_extractor = Model(inputs=model.input, outputs=model.layers[-6].output)  # Extract features before the Dense layer
    return feature_extractor

# Train the original model
input_shape = (224, 224, 3)  # Modify this according to your input image dimensions
num_classes = 10  # Modify according to the number of classes in your dataset

# Assuming train_data, val_data, test_data are already defined
# Example: train_data = (images_train, labels_train), val_data = (images_val, labels_val)
model = create_model(input_shape, num_classes)
model.fit(train_data, epochs=10, validation_data=val_data)

# Evaluate the model (optional)
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

# Feature extraction
feature_extractor = create_feature_extractor(input_shape)
features = feature_extractor.predict(train_data[0])  # Extract features from train_data

# Flatten the features (if necessary)
features_flattened = features.reshape(features.shape[0], -1)  # Flatten the features to 2D

# Apply K-Means Clustering
n_clusters = 5  # You can adjust the number of clusters
kmeans = KMeans(n_clusters=n_clusters)
cluster_labels = kmeans.fit_predict(features_flattened)

# Print the clustering results
print("Cluster labels for the first 10 samples:", cluster_labels[:10])

# Evaluate the clustering using silhouette score (higher is better)
silhouette_avg = silhouette_score(features_flattened, cluster_labels)
print(f"Silhouette Score: {silhouette_avg:.4f}")

In [4]:
#Hybrid

class SelfAttention(Layer):
    def __init__(self, embed_dim):
        super(SelfAttention, self).__init__()
        self.query_dense = Dense(embed_dim)
        self.key_dense = Dense(embed_dim)
        self.value_dense = Dense(embed_dim)
        self.softmax = Softmax(axis=-1)

    def call(self, inputs):
        Q = self.query_dense(inputs)
        K = self.key_dense(inputs)
        V = self.value_dense(inputs)

        attention_scores = tf.matmul(Q, K, transpose_b=True)
        attention_weights = self.softmax(attention_scores)

        return tf.matmul(attention_weights, V)
    
def build_hybrid_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3,3), activation="relu", padding="same")(inputs)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(128, (3,3), activation="relu", padding="same")(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(128, activation="relu")(x)
    x = Reshape((1, 128))(x)

    x = SelfAttention(128)(x)
    x = Flatten()(x)

    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs, outputs)
    return model

model = build_hybrid_model(input_shape, num_classes)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
72/72 [==============================] - 18s 237ms/step - loss: 1.3057 - accuracy: 0.3592 - val_loss: 1.3261 - val_accuracy: 0.3857
Epoch 2/10
72/72 [==============================] - 17s 235ms/step - loss: 1.1756 - accuracy: 0.4697 - val_loss: 1.4444 - val_accuracy: 0.2897
Epoch 3/10
72/72 [==============================] - 17s 239ms/step - loss: 1.1158 - accuracy: 0.5007 - val_loss: 1.4028 - val_accuracy: 0.3019
Epoch 4/10
72/72 [==============================] - 17s 236ms/step - loss: 1.0367 - accuracy: 0.5472 - val_loss: 1.5175 - val_accuracy: 0.3281
Epoch 5/10
72/72 [==============================] - 17s 240ms/step - loss: 0.9680 - accuracy: 0.5812 - val_loss: 1.4198 - val_accuracy: 0.3892
Epoch 6/10
72/72 [==============================] - 18s 250ms/step - loss: 0.9843 - accuracy: 0.5655 - val_loss: 1.3011 - val_accuracy: 0.3979
Epoch 7/10
72/72 [==============================] - 19s 263ms/step - loss: 0.9016 - accuracy: 0.6160 - val_loss: 1.4520 - val_accuracy: 0.4328

In [5]:
#DenseNet121

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers[:-31]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation="softmax")(x)    

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
72/72 [==============================] - 26s 290ms/step - loss: 0.9109 - accuracy: 0.6374 - val_loss: 0.7716 - val_accuracy: 0.6736
Epoch 2/10
72/72 [==============================] - 19s 259ms/step - loss: 0.4819 - accuracy: 0.8241 - val_loss: 0.5768 - val_accuracy: 0.7696
Epoch 3/10
72/72 [==============================] - 20s 270ms/step - loss: 0.3704 - accuracy: 0.8694 - val_loss: 0.4970 - val_accuracy: 0.8150
Epoch 4/10
72/72 [==============================] - 19s 265ms/step - loss: 0.2938 - accuracy: 0.8938 - val_loss: 0.3825 - val_accuracy: 0.8656
Epoch 5/10
72/72 [==============================] - 19s 257ms/step - loss: 0.2380 - accuracy: 0.9208 - val_loss: 0.3974 - val_accuracy: 0.8569
Epoch 6/10
72/72 [==============================] - 18s 253ms/step - loss: 0.2238 - accuracy: 0.9116 - val_loss: 0.4078 - val_accuracy: 0.8569
Epoch 7/10
72/72 [==============================] - 19s 258ms/step - loss: 0.1691 - accuracy: 0.9434 - val_loss: 0.4149 - val_accuracy: 0.8586

In [6]:
#ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers[:140]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
72/72 [==============================] - 24s 278ms/step - loss: 1.1344 - accuracy: 0.4980 - val_loss: 1.5019 - val_accuracy: 0.2862
Epoch 2/10
72/72 [==============================] - 18s 252ms/step - loss: 0.9914 - accuracy: 0.5629 - val_loss: 1.3836 - val_accuracy: 0.3002
Epoch 3/10
72/72 [==============================] - 18s 254ms/step - loss: 0.9202 - accuracy: 0.6173 - val_loss: 1.1741 - val_accuracy: 0.4188
Epoch 4/10
72/72 [==============================] - 19s 258ms/step - loss: 0.8912 - accuracy: 0.6243 - val_loss: 1.0933 - val_accuracy: 0.5236
Epoch 5/10
72/72 [==============================] - 19s 260ms/step - loss: 0.8193 - accuracy: 0.6622 - val_loss: 1.0952 - val_accuracy: 0.5602
Epoch 6/10
72/72 [==============================] - 19s 260ms/step - loss: 0.7941 - accuracy: 0.6626 - val_loss: 1.0679 - val_accuracy: 0.5689
Epoch 7/10
72/72 [==============================] - 18s 253ms/step - loss: 0.8037 - accuracy: 0.6565 - val_loss: 1.8302 - val_accuracy: 0.4241

In [7]:
#EfficientNetB0

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers[:-50]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=val_data)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10
72/72 [==============================] - 23s 255ms/step - loss: 1.3929 - accuracy: 0.2860 - val_loss: 1.4047 - val_accuracy: 0.2880
Epoch 2/10
72/72 [==============================] - 17s 240ms/step - loss: 1.3622 - accuracy: 0.2978 - val_loss: 1.3980 - val_accuracy: 0.2862
Epoch 3/10
72/72 [==============================] - 18s 242ms/step - loss: 1.3546 - accuracy: 0.3047 - val_loss: 1.3613 - val_accuracy: 0.2862
Epoch 4/10
72/72 [==============================] - 18s 243ms/step - loss: 1.3484 - accuracy: 0.2912 - val_loss: 1.3621 - val_accuracy: 0.2862
Epoch 5/10
72/72 [==============================] - 17s 239ms/step - loss: 1.3330 - accuracy: 0.2995 - val_loss: 1.4096 - val_accuracy: 0.2862
Epoch 6/10
72/72 [==============================] - 17s 240ms/step - loss: 1.3209 - accuracy: 0.3239 - val_loss: 1.3492 - val_accuracy: 0.2862
Epoch 7/10
72/72 [==============================] - 17s 240ms/step - loss: 1.3066 - accuracy: 0.3287 - val_loss: 1.4567 - val_accuracy: 0.3403